In [1]:
from numpy import random

def rule_one_possible(string):
    
    # If the last letter of the current string is I, rule 1 can be applied
    
    if string[-1:] == 'I':
        return True
    
    else:
        return False


def rule_two_possible(string):
    
    # If the first letter of the current string is M, rule 2 can be applied
    
    if string[:1] == 'M':
        return True
    else:
        return False


def rule_three_possible(string):
    
    # If the current string contains III, rule 3 can be applied
     
    if string.find('III') != -1:
        return True
    else:
        return False
    
    
def rule_four_possible(string):
    
    # If the current string contains UU, rule 4 can be applied
    
    if string.find('UU') != -1:
        return True
    else:
        return False    
    
    
def rule_three_options(string):
    
    indices = [index for index, char in enumerate(string) if string.startswith('III', index)]
    
    return indices
    
    
def rule_four_options(string):
    
    indices = [index for index, char in enumerate(string) if string.startswith('UU', index)]
    
    return indices

    
def get_options(string):

    options = []

    if rule_one_possible(string):
        options.append('apply_rule_one(string)')

    if rule_two_possible(string):
        options.append('apply_rule_two(string)')

    if rule_three_possible(string):
        for option in rule_three_options(string):
            options.append(f'apply_rule_three(string, {option})')
        
    if rule_four_possible(string):
        for option in rule_four_options(string):
            options.append(f'apply_rule_four(string, {option})')

    return options





def apply_rule_one(string):
    
    return string + 'U'


def apply_rule_two(string):
    
    return string + string[1:]


def apply_rule_three(string, index):
    
    return string[:index] + 'U' + string[index+3:]


def apply_rule_four(string, index):
    
    return string[:index] + string[index+2:]



def apply_random_option(options):
    
    choice = random.choice(options)
    
    eval(choice)



In [3]:
get_options('MIIUUUIIUIIIUUUUUIUUUIIIIIUUIUUUIUUIIUI')

['apply_rule_one(string)',
 'apply_rule_two(string)',
 'apply_rule_three(string, 9)',
 'apply_rule_three(string, 21)',
 'apply_rule_three(string, 22)',
 'apply_rule_three(string, 23)',
 'apply_rule_four(string, 3)',
 'apply_rule_four(string, 4)',
 'apply_rule_four(string, 12)',
 'apply_rule_four(string, 13)',
 'apply_rule_four(string, 14)',
 'apply_rule_four(string, 15)',
 'apply_rule_four(string, 18)',
 'apply_rule_four(string, 19)',
 'apply_rule_four(string, 26)',
 'apply_rule_four(string, 29)',
 'apply_rule_four(string, 30)',
 'apply_rule_four(string, 33)']

In [161]:
import pandas as pd


def apply_rules_randomly(string, num_applications):

    i=0
    while i < num_applications:

        options = get_options(string)

        choice = random.choice(options)

        string = eval(choice)
        
        print(string in endpoints)
    
        i+=1

    return string

In [ ]:

df = pd.DataFrame(columns=['string'])

for i in range(0, 1000):
    
    endstring = apply_rules_randomly('MI', 4)    # 48 possible end states
    
    df.loc[len(df)] = endstring


print(df['string'].value_counts())
print(df['string'].nunique())

In [206]:
apply_rules_randomly('MI', num_applications=5) 

False
False
False
False
True


'MIIUIIUIIUIIUIIUIIUIIUIIU'

## problem exploration for maximum of 5 successive moves

In [ ]:
"""
Result of:

>>> for i in range(0, 100090):   
>>>    endstring = apply_rules_randomly('MI', 5)
    

MIUIUIUIUIUIUIUIUIUIUIUIUIUIUIUIU    50274
MIIUIIUIIUIIUIIUIIUIIUIIU            25021
MIUIUIUIU                             6256
MUIUUIU                               5242
MUIUIU                                1550
MUIUIUIUI                             1547
MIUUIUU                               1031
MI                                     994
MIUUIIIIU                              430
MUIUIIIIU                              430
MIUIU                                  425
MIIIIUIIIIUIIIIUIIIIU                  420
MIIIIUUIU                              419
MIIUU                                  410
MIUUI                                  389
MIIIUIIU                               386
MIIUIIIU                               383
MUUII                                  383
MIIIIUIUU                              369
MUIUI                                  339
MUIIU                                  330
MIUIIIIU                               311
MIIIIUIU                               298
MIIIUIIIIIUII                          293
MIIUIIIIIUIII                          269
MUIIIIIU                               247
MIIIIIUIIIIIU                          193
MIIIIUIIIIIUI                          186
MIUIIIIIUIIII                          160
MUIIIIIUIIIII                          150
MIIIIIIIIUIIIIIIIIU                    110
MIIIIIUU                               102
MIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIII       56
MIIIIIIIIIIIIIIIIU                      55
MIIIIIIIIIIIIUI                         55
MIIIIIUIIIIIIII                         54
MIIIIIIUIIIIIII                         51
MIIIIIIIIUIIIII                         51
MIIIIIIIIIIIIIU                         50
MIIIIIIIIIIUIII                         46
MIIIIIIIIIUIIII                         46
MIIIIIIIIIIIUII                         44
MIIIIIIIUIIIIII                         43
MIUIIIIIIIIIIII                         42
MIIUIIIIIIIIIII                         41
MIIIIUIIIIIIIII                         38
MUIIIIIIIIIIIII                         38
MIIIUIIIIIIIIII                         33
"""


df = pd.DataFrame(columns=['string'])

for i in range(0, 1000):
    
    endstring = apply_rules_randomly('MI', 5)    # 48 possible end states
    
    df.loc[len(df)] = endstring


print(df['string'].value_counts())
print(df['string'].nunique())

In [159]:
endpoints = ['MIUIUIUIUIUIUIUIUIUIUIUIUIUIUIUIU',
'MIIUIIUIIUIIUIIUIIUIIUIIU',
'MIUIUIUIU',
'MUIUUIU',
'MUIUIU',
'MUIUIUIUI',
'MIUUIUU',
'MI',
'MIUUIIIIU',
'MUIUIIIIU',
'MIUIU',
'MIIIIUIIIIUIIIIUIIIIU',
'MIIIIUUIU',
'MIIUU',
'MIUUI',
'MIIIUIIU',
'MIIUIIIU',
'MUUII',
'MIIIIUIUU',
'MUIUI',
'MUIIU',
'MIUIIIIU',
'MIIIIUIU',
'MIIIUIIIIIUII',
'MIIUIIIIIUIII',
'MUIIIIIU',
'MIIIIIUIIIIIU',
'MIIIIUIIIIIUI',
'MIUIIIIIUIIII',
'MUIIIIIUIIIII',
'MIIIIIIIIUIIIIIIIIU',
'MIIIIIUU',
'MIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIII',
'MIIIIIIIIIIIIIIIIU',
'MIIIIIIIIIIIIUI',
'MIIIIIUIIIIIIII',
'MIIIIIIUIIIIIII',
'MIIIIIIIIUIIIII',
'MIIIIIIIIIIIIIU',
'MIIIIIIIIIIUIII',
'MIIIIIIIIIUIIII',
'MIIIIIIIIIIIUII',
'MIIIIIIIUIIIIII',
'MIUIIIIIIIIIIII',
'MIIUIIIIIIIIIII',
'MIIIIUIIIIIIIII',
'MUIIIIIIIIIIIII',
'MIIIUIIIIIIIIII']

In [ ]:

#  def rule_one_options(string):
#      
#      if rule_one_possible(string) == True:
#      
#          return [len(string)]
#      
#      else:
#          return []
#  
#  
#  def rule_two_options(string):
#      
#      if rule_two_possible(string) == True:
#          
#          return [1]
#      
#      else:
#          return []

